In [8]:
import math
import os
import requests
from PIL import Image
from io import BytesIO

# Set a higher limit for image size
Image.MAX_IMAGE_PIXELS = 10000000000  # Adjust as needed, this is an example limit
def resize_image(image, max_size=(5000, 5000)):  # Example max size
    if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
        image = image.resize(max_size, Image.ANTIALIAS)
    return image

# def lat_lon_to_tile(lat, lon, zoom):
#     lat_rad = math.radians(lat)
#     n = 2.0 ** zoom
#     x = int((lon + 180.0) / 360.0 * n)
#     y = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
#     return x, y

def lat_lon_to_tile(lat, lon, zoom):
    lat_rad = math.radians(lat)
    n = 2.0 ** zoom
    x = int((lon + 180.0) / 360.0 * n)
    
    y = int((1.0 - math.log(math.tan(lat_rad) + 1 / math.cos(lat_rad)) / math.pi) / 2.0 * n)
    
    print(f"lat: {lat}, lon: {lon} -> x: {x}, y: {y} (zoom: {zoom})")
    return x, y

def get_tile_range(min_lat, max_lat, min_lon, max_lon, zoom):
    min_x, min_y = lat_lon_to_tile(min_lat, min_lon, zoom)
    max_x, max_y = lat_lon_to_tile(max_lat, max_lon, zoom)
    return (min_x, max_x), (max_y, min_y)  # Ensure the Y range is in the correct order

# def fetch_tile(url, output_image_path):
#     if(os.path.isfile(output_image_path)): return
#     print("Fetching tile url: ",url)
#     response = requests.get(url)
#     if response.status_code == 200:
#         img = Image.open(BytesIO(response.content))
#         img.save(output_image_path)
#         img.close()
#     else:
#         print(f"Failed to fetch the tile. Status code: {response.status_code}")

def fetch_tile(url, output_image_path):
    if os.path.isfile(output_image_path): 
        return
    print("Fetching tile url: ", url)
    response = requests.get(url)
    if response.status_code == 200:
        try:
            img = Image.open(BytesIO(response.content))
            img.save(output_image_path)
            img.close()  # Ensure the image file is closed
        except Exception as e:
            print(f"Error processing tile {url}: {e}")
    else:
        print(f"Failed to fetch the tile. Status code: {response.status_code}")

# def fetch_tiles_for_area(min_lat, max_lat, min_lon, max_lon, zoom, base_url, output_dir):
#     tile_x_range, tile_y_range = get_tile_range(min_lat, max_lat, min_lon, max_lon, zoom)
#     width = (tile_x_range[1] - tile_x_range[0] + 1) * 256
#     height = (tile_y_range[1] - tile_y_range[0] + 1) * 256
#     print(f"Expected image resolution: {width} x {height}")
#     for x in range(tile_x_range[0], tile_x_range[1] + 1):
#         for y in range(tile_y_range[0], tile_y_range[1] + 1):
#             # tile_url = f"{base_url}&TileMatrix=EPSG:3857:{zoom}&TileCol={x}&TileRow={y}"
#             tile_url = f"{base_url}/{zoom}/{x}/{y}.png"
#             output_image_path = f"{output_dir}/tile_{zoom}_{x}_{y}.png"
#             fetch_tile(tile_url, output_image_path)

def fetch_tiles_for_area(min_lat, max_lat, min_lon, max_lon, zoom, base_url, output_dir, batch_size=10):
    tile_x_range, tile_y_range = get_tile_range(min_lat, max_lat, min_lon, max_lon, zoom)
    for i, x in enumerate(range(tile_x_range[0], tile_x_range[1] + 1)):
        if i % batch_size == 0:
            # Optional: Close or process any resources, if necessary
            print(f"Processed {i} tiles, starting a new batch...")
        
        for y in range(tile_y_range[0], tile_y_range[1] + 1):
            tile_url = f"{base_url}/{zoom}/{x}/{y}.png"
            output_image_path = f"{output_dir}/tile_{zoom}_{x}_{y}.png"
            fetch_tile(tile_url, output_image_path)

# def stitch_tiles(output_dir, tile_x_range, tile_y_range, zoom):
#     tiles = {}
#     for x in range(tile_x_range[0], tile_x_range[1] + 1):
#         for y in range(tile_y_range[0], tile_y_range[1] + 1):
#             tile_path = f"{output_dir}/tile_{zoom}_{x}_{y}.png"
#             if os.path.exists(tile_path):
#                 tiles[(x, y)] = Image.open(tile_path)
    
#     tile_size = list(tiles.values())[0].size if tiles else (256, 256) # Default to 256x256 if no tiles are present
#     print("tile_size for stitching",tile_size)
#     stitched_image = Image.new('RGB', ((tile_x_range[1] - tile_x_range[0] + 1) * tile_size[0],
#                                        (tile_y_range[1] - tile_y_range[0] + 1) * tile_size[1]))

#     for (x, y), tile in tiles.items():
#         stitched_image.paste(tile, ((x - tile_x_range[0]) * tile_size[0], (y - tile_y_range[0]) * tile_size[1]))
    
#     return stitched_image

def stitch_tiles(output_dir, tile_x_range, tile_y_range, zoom):
    tile_size = (256, 256)
    stitched_image = Image.new('RGB', ((tile_x_range[1] - tile_x_range[0] + 1) * tile_size[0],
                                       (tile_y_range[1] - tile_y_range[0] + 1) * tile_size[1]))

    for x in range(tile_x_range[0], tile_x_range[1] + 1):
        for y in range(tile_y_range[0], tile_y_range[1] + 1):
            tile_path = f"{output_dir}/tile_{zoom}_{x}_{y}.png"
            if os.path.exists(tile_path):
                tile = Image.open(tile_path)
                stitched_image.paste(tile, ((x - tile_x_range[0]) * tile_size[0], (y - tile_y_range[0]) * tile_size[1]))
                tile.close()  # Close the tile to release memory
    return stitched_image

# def crop_image_to_bbox(image, min_lat, max_lat, min_lon, max_lon, zoom):
#     print(f"image_size: {image.size[0]}x{image.size[1]}")  # 8192x8192
    
#     # Get the tile coordinates for min and max lat, lon
#     min_x, min_y = lat_lon_to_tile(min_lat, min_lon, zoom)
#     max_x, max_y = lat_lon_to_tile(max_lat, max_lon, zoom)

#     # Calculate the number of tiles in the x and y directions
#     num_tiles_x = max_x - min_x + 1
#     num_tiles_y = max_y - min_y + 1
#     print(f"Tiles in x: {num_tiles_x}, Tiles in y: {num_tiles_y}")

#     # Calculate the size of each tile in pixels
#     tile_width = image.size[0] // (max_x - min_x + 1)
#     tile_height = image.size[1] // (max_y - min_y + 1)
#     print(f"tile_width: {tile_width}, tile_height: {tile_height}")

#     # Calculate the cropping box (in pixels)
#     left = (min_x - min_x) * tile_width
#     upper = (min_y - min_y) * tile_height
#     right = (max_x - min_x + 1) * tile_width
#     lower = (max_y - min_y + 1) * tile_height

#     print(f"Cropping box: left={left}, upper={upper}, right={right}, lower={lower}")

#     # Crop and return the image
#     return image.crop((left, upper, right, lower))

def crop_image_to_bbox(image, min_lat, max_lat, min_lon, max_lon, zoom):
    print(f"image_size: {image.size[0]}x{image.size[1]}")  # 8192x8192
    
    # Get the tile coordinates for min and max lat, lon
    min_x, min_y = lat_lon_to_tile(min_lat, min_lon, zoom)
    max_x, max_y = lat_lon_to_tile(max_lat, max_lon, zoom)

    # Ensure correct min_y and max_y
    if min_y > max_y:
        min_y, max_y = max_y, min_y

    print(f"Tiles in x: {max_x - min_x + 1}, Tiles in y: {max_y - min_y + 1}")

    # Calculate the size of each tile in pixels
    tile_width = image.size[0] // (max_x - min_x + 1)
    tile_height = image.size[1] // (max_y - min_y + 1)
    print(f"tile_width: {tile_width}, tile_height: {tile_height}")

    # Calculate the cropping box (in pixels)
    left = (min_x - min_x) * tile_width
    upper = (min_y - min_y) * tile_height
    right = (max_x - min_x + 1) * tile_width
    lower = (max_y - min_y + 1) * tile_height

    print(f"Cropping box: left={left}, upper={upper}, right={right}, lower={lower}")

    # Crop and return the image
    return image.crop((left, upper, right, lower))


# Example usage
min_lat, max_lat = 34.07201, 34.21606
min_lon, max_lon = 77.45396, 77.62802
zoom = 17
# base_url = "https://services.terrascope.be/wmts/v2?layer=WORLDCOVER_2021_MAP&style=&tilematrixset=EPSG:3857&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image/png&TIME=2025-01-20"
base_url = "http://192.168.1.130:9002/api/esa"
output_dir = "./new_tiles"

fetch_tiles_for_area(min_lat, max_lat, min_lon, max_lon, zoom, base_url, output_dir)

tile_x_range, tile_y_range = get_tile_range(min_lat, max_lat, min_lon, max_lon, zoom)
stitched_image = stitch_tiles(output_dir, tile_x_range, tile_y_range, zoom)
# stitched_image = resize_image(stitched_image)
cropped_image = crop_image_to_bbox(stitched_image, min_lat, max_lat, min_lon, max_lon, zoom)

cropped_image.save("stitched_cropped_image_17_.png")


lat: 33.60925513738903, lon: 75.79202885747709 -> x: 11641, y: 6566 (zoom: 14)
lat: 33.65890841019596, lon: 75.90898450959517 -> x: 11646, y: 6563 (zoom: 14)
tile_x_range, tile_y_range (11641, 11646) (6563, 6566)
Processed 0 tiles, starting a new batch...
Fetching tile url:  https://services.terrascope.be/wmts/v2?layer=WORLDCOVER_2021_MAP&style=&tilematrixset=EPSG:3857&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image/png&TIME=2025-01-20&TileMatrix=EPSG:3857:14&TileCol=11641&TileRow=6563
Fetching tile url:  https://services.terrascope.be/wmts/v2?layer=WORLDCOVER_2021_MAP&style=&tilematrixset=EPSG:3857&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image/png&TIME=2025-01-20&TileMatrix=EPSG:3857:14&TileCol=11641&TileRow=6564
Fetching tile url:  https://services.terrascope.be/wmts/v2?layer=WORLDCOVER_2021_MAP&style=&tilematrixset=EPSG:3857&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image/png&TIME=2025-01-20&TileMatrix=EPSG:3857:14&TileCol=11641&TileRow=6565
Fetching tile u

c:\Users\THREE\Desktop\Blosm\.venv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
